In [1]:
import  requests 
import time
import pandas as pd

from urllib.error import URLError, HTTPError
from urllib3.exceptions import SSLError

from requests.exceptions import HTTPError
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from socket import timeout

In [2]:
search_input = 'Turkish Airlines'
search_string = '+'.join(str(e) for e in search_input.lower().split())

## each search page has number of results pages with a limit of 50 news article in each one.

url1 = f'https://www.dailysabah.com/search?query={search_string}&qlimit=by_fifty&pgno='
url2 = f'https://www.dailysabah.com/search?query={search_string}&qlimit=by_fifty&qsort=oldest&pgno='
urls = [url1,url2]
print(url1)
print(url2)


https://www.dailysabah.com/search?query=turkish+airlines&qlimit=by_fifty&pgno=
https://www.dailysabah.com/search?query=turkish+airlines&qlimit=by_fifty&qsort=oldest&pgno=


In [3]:
list_of_links = []

for url in urls:
    numbers = 1
    try: 
    
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
        reg_url = url
        req = Request(url=reg_url, headers=headers) 
        html = urlopen(req).read()  

        soup = BeautifulSoup(html, "html.parser")
    
        data = soup.findAll('div',{'class':'pages_list'})
        data2 = str(data).split('pgno=')
        number = int(data2[-2].split('">')[0])
        
        for i in range(1,number+1,1):
            list_of_links.append(url+str(i))
            
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
        pass
    except Exception as err:
        print(f'Other1 error occurred: {err}')  
        pass

    
print(len(list_of_links))
print(list_of_links[0])
print(list_of_links[-1])

98
https://www.dailysabah.com/search?query=turkish+airlines&qlimit=by_fifty&pgno=1
https://www.dailysabah.com/search?query=turkish+airlines&qlimit=by_fifty&qsort=oldest&pgno=49


In [28]:
headlines = []
for url in list_of_links:
    try: 
    
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
        reg_url = url
        req = Request(url=reg_url, headers=headers) 
        html = urlopen(req).read()  

        soup = BeautifulSoup(html, "html.parser")
    
        data = soup.findAll('ul',{'class':'items_list'})
        data2 = str(data[0]).split('<div class="image_holder">')

        for i in range(1,len(data2),2):
            link = data2[i].split('<a href="')[1].split('">')[0]
            date =  data2[i].split('<div class="date_text">')[1].split('</div')[0].strip()
            headline = data2[i].split('<h3>')[1].split('</a>')[0].split('">')[1].strip()        
            headlines.append([headline,date,link])
            
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
        pass
    except Exception as err:
        print(f'Other1 error occurred: {err}')  
        pass


In [35]:
print(headlines[-3])

print(len(headlines))

['Turkish Airlines offers flights to Tajikistan', 'Aug 04, 2015', 'https://www.dailysabah.com/business/2015/08/04/turkish-airlines-offers-flights-to-tajikistan']
2425


In [36]:
    
df = pd.DataFrame(headlines, columns =['text','date','article_link'])
df.to_csv(f'apis_datasets/{search_input}_DailySabah.csv', encoding='utf-8', index=False)

df.head()  
df.describe

<bound method NDFrame.describe of                                                    text          date  \
0     Turkey virtually halts intercity travel amid C...  Mar 27, 2020   
1     Turkey Wealth Fund inks $5B MoU with China's S...  Mar 26, 2020   
2     Anti-virus fight: More test kits arrive as dea...  Mar 23, 2020   
3     THY's Turkish Cargo to open route to Linz, Aus...  Mar 23, 2020   
4     Turkish Airlines grounding all intl flights, e...  Mar 22, 2020   
...                                                 ...           ...   
2420  Turkish Airlines plane makes emergency landing...  Jul 30, 2015   
2421  Turan shows off-field leadership during Barcel...  Jul 31, 2015   
2422      Turkish Airlines offers flights to Tajikistan  Aug 04, 2015   
2423  Foreign net sales in Turkey valued at $248 mil...  Aug 07, 2015   
2424  Kenya seeks to boost trade with Turkey, says K...  Aug 08, 2015   

                                           article_link  
0     https://www.dailysabah.co